In [12]:
from unittest.mock import Mock, patch
from pytest import raises

## patch.object

If we're working directly with an object, we can use `patch.object`

In [25]:
class User:
    def is_birthday(self):
        pass
        
    def greet(self):
        if self.is_birthday():
            return 'happy birthday'
        else:
            return 'hello'

user = User()
with patch.object(user, 'is_birthday', return_value=True):
    # user.is_birthday is a MagicMock that returns True
    print(user.is_birthday, user.is_birthday())
    # Check the outcome
    assert user.greet() == 'happy birthday'

<MagicMock name='is_birthday' id='139676488238248'> True


However, we don't always have direct access to that object.

## patch

In [14]:
import requests

# a MagicMock is returned by the patched method
with patch('requests.get'):
    print(requests.get)

<MagicMock name='get' id='139676488495680'>


In [15]:
# Example of a function that might need patching
import github3

def get_followers(username):
    api_client = github3.GitHub()
    return api_client.user(username).followers

### decorator

In [16]:
# Function decorator
@patch('github3.GitHub')
def test_get_followers(mockGitHub):
    mockGitHub.return_value.user.return_value.followers = 20
    assert get_followers('helenst') == 20
    
test_get_followers()  # succeeds

In [17]:
# Class decorator - if all your test methods want to mock the same thing
@patch('github3.GitHub')
class GitHubTest:
    def test_get_followers(mockGitHub):
        mockGitHub.return_value.user.return_value.followers = 20
        assert get_followers('helenst') == 20  # succeeds
    
GitHubTest.test_get_followers()

### context manager

In [18]:
with patch('github3.GitHub') as mockGitHub:
    mockGitHub.return_value.user.return_value.followers = 20
    assert get_followers('helenst') == 20

### manual patch control

In [19]:
# Use start and stop for more control
patcher = patch('github3.GitHub')
mockGitHub = patcher.start()
mockGitHub.return_value.user.return_value.followers = 20
assert get_followers('helenst') == 20  # succeeds
patcher.stop()

## Where to patch?

In [20]:
## file: github_utils
#
# import github3
#
# def get_followers(username):
#    api_client = github3.GitHub()
#    return api_client.user(username).followers

import github_utils

with patch('github3.GitHub') as mockGitHub:
    mockGitHub.return_value.user.return_value.followers = 200
    assert github_utils.get_followers('helenst') == 200

In [21]:
## file: github_utils2
#
# from github3 import GitHub
#
# def get_followers(username):
#    api_client = GitHub()
#    return api_client.user(username).followers

import github_utils2
    
with patch('github3.GitHub') as mockGitHub:
    mockGitHub.return_value.user.return_value.followers = 200
    
    with raises(AssertionError):
        assert github_utils2.get_followers('helenst') == 200

In [22]:
import github_utils2

# "ensure that you patch the name used by the system under test" - the python docs
with patch('github_utils2.GitHub') as mockGitHub:
    mockGitHub.return_value.user.return_value.followers = 200
    assert github_utils2.get_followers('helenst') == 200